<a href="https://colab.research.google.com/github/priyanshu7466/Python/blob/main/StockMarket/WiDS_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import models, datasets, layers
from keras.datasets import mnist
from matplotlib import pyplot as plt
import numpy as np
import random
from tvDatafeed import TvDatafeed, Interval
import pandas as pd
import talib as tb
import time
import os
import matplotlib.pyplot as plt

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')

In [ ]:
n50_symbols=np.array(['NIFTY', 'HINDALCO', 'MARUTI', 'NESTLEIND', 'ONGC', 'TATAMOTORS', 'ITC', 'SUNPHARMA', 'BHARTIARTL', 'CIPLA', 'DIVISLAB', 'JSWSTEEL', 'HINDUNILVR', 'TITAN', 'NTPC', 'TATASTEEL', 'HEROMOTOCO', 'HDFCLIFE', 'ULTRACEMCO', 'SBILIFE', 'TCS', 'BAJFINANCE', 'BRITANNIA', 'AXISBANK', 'TECHM', 'COALINDIA', 'LT', 'DRREDDY', 'M_M', 'APOLLOHOSP', 'ADANIPORTS', 'BAJAJFINSV', 'GRASIM', 'TATACONSUM', 'ICICIBANK', 'HDFCBANK', 'EICHERMOT', 'BPCL', 'ADANIENT', 'UPL', 'POWERGRID', 'HDFCBANK', 'INFY', 'WIPRO', 'RELIANCE', 'INDUSINDBK', 'ASIANPAINT', 'HCLTECH', 'KOTAKBANK', 'BAJAJ_AUTO', 'SBIN'])

folder_name = 'Daily_historical_data'

X_train=[]
y_train=[]
y_train_pos=[]
gain=[]


for i in range(51):
    file_name = f'{n50_symbols[i]}.csv'
    file_path = os.path.join('/content/drive/MyDrive', folder_name, file_name)
    df=pd.read_csv(file_path)
    df['datetime']=pd.to_datetime(df['datetime'])
    df['year'] = df['datetime'].dt.year
    df['hour'] = df['datetime'].dt.hour
    df['minute'] = df['datetime'].dt.minute

    ###############################Indicators#######################

    #ema of last 5 candle changes
    df['ema200']=tb.EMA(df['close'],200)
    #aroon
    df["aroon"]=tb.AROONOSC(df['high'],df['low'],timeperiod=14)
    # #macd
    df["macd"]=tb.MACD(df['close'], fastperiod=12, slowperiod=26, signalperiod=9)[0]
    #bbands
    df['bbands_up']=tb.BBANDS(df['close'],5,2,2,0)[0]
    df['bbands_mid']=tb.BBANDS(df['close'],5,2,2,0)[1]
    df['bbands_down']=tb.BBANDS(df['close'],5,2,2,0)[2]
    #rsi
    df["rsi"]=tb.RSI(df['close'])

    df=df.iloc[201:].reset_index(drop=True)
    # print(df)

    start_ind=[]
    for j in range(len(df.index)):
        if df['hour'][j]==9 and df['minute'][j]==15: # Getting the indices where the trading day starts
            start_ind.append(j)
    end_ind=[] # end of the day indices
    for j in range(1,len(start_ind)): #starting from 1 because we dont want an end without the start of a day which is the case for start_of_day[0]-1
        end_ind.append(start_ind[j]-1)

    # print(start_ind)
    # print(end_ind)

    if len(start_ind)>len(end_ind):
        start_ind=start_ind[:-1]

    # print(start_ind)
    # print(end_ind)
    j=0
    while j<len(start_ind):
        if end_ind[j]-start_ind[j]!=74:
            end_ind.remove(end_ind[j])
            start_ind.remove(start_ind[j])
            continue

        # X from 9:25AM to 3:15pm
        # y from 3:15 to 9:45am the next day
        df_new=df.iloc[start_ind[j]+2:end_ind[j]-2].drop(['symbol','datetime','year','hour','minute'],axis=1)

        ###################         Imputer          #####################
        df_new = pd.DataFrame(imputer.fit_transform(df_new))

        if len(df_new.columns)==len(df.columns)-5:
            X_train.append(df_new.values.flatten())
        else:
            end_ind.remove(end_ind[j])
            start_ind.remove(start_ind[j])

        j+=1

    print(i)

    for j in end_ind:
        gain_inst=(df['close'][j+7]-df['close'][j-2])*100/df['close'][j-2] # gain at the particular instant
        gain.append(gain_inst)  # i+2 indicates the next day at 9:20(as we need to let the price settle) and i-2 at time 3:15 as we need to trade before the day ends and we cant trade ecaxtly at the end of the day
        if gain_inst>=0.1:
            y_train_pos.append(1)
        elif gain_inst<-0.1:
            y_train_pos.append(-1)
        else:
            y_train_pos.append(0)

y_train=gain

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50


In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Normalizing the data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)

In [ ]:
from sklearn.svm import SVR
from sklearn.utils import shuffle

#shuffled data
X_train_shuffled, y_train_shuffled = shuffle(X_train, y_train, random_state=42)

svr_regressor=SVR(kernel='sigmoid',gamma='auto',C=1)

svr_regressor.fit(X_train_shuffled,y_train_shuffled)

SVR(C=1, gamma='auto', kernel='sigmoid')

In [ ]:
y_predicted=svr_regressor.predict(X_train)

y_pred_pos=[]
for y in y_predicted:
    if y>0.1:
        y_pred_pos.append(1)
    elif y<-0.1:
        y_pred_pos.append(-1)
    else:
        y_pred_pos.append(0)


In [ ]:
tv = TvDatafeed()

x=1

while x==1:

    # df50_init=[]
    X_pred=[]
    y_pred=[]
    y_pos=[]
    price_arr=[]


    for i in range(51):
        data_5_min = tv.get_hist(symbol=f'{n50_symbols[i]}',exchange='NSE',interval=Interval.in_5_minute,n_bars=500)
        df=pd.DataFrame(data_5_min)
        df=df.reset_index()
        df['year'] = df['datetime'].dt.year
        df['hour'] = df['datetime'].dt.hour
        df['minute'] = df['datetime'].dt.minute

            ###############################Indicators#######################

        #ema of last 5 candle changes
        df['ema200']=tb.EMA(df['close'],200)
        #aroon
        df["aroon"]=tb.AROONOSC(df['high'],df['low'],timeperiod=14)
        # #macd
        df["macd"]=tb.MACD(df['close'], fastperiod=12, slowperiod=26, signalperiod=9)[0]
        #bbands
        df['bbands_up']=tb.BBANDS(df['close'],5,2,2,0)[0]
        df['bbands_mid']=tb.BBANDS(df['close'],5,2,2,0)[1]
        df['bbands_down']=tb.BBANDS(df['close'],5,2,2,0)[2]
        #rsi
        df["rsi"]=tb.RSI(df['close'])

        df=df.iloc[201:].reset_index(drop=True)


        df_for_pred=df.iloc[-70:].drop(['symbol','datetime','year','hour','minute'],axis=1)
        #print(df_for_pred)
        X_pred.append(df_for_pred.values.flatten())

        # Price of stock at that instant
        price_arr.append(df['close'][len(df.index)-1])

    X_pred_norm=scaler.transform(X_pred)
    y_pred=svr_regressor.predict(X_pred_norm)
    #print(y_pred)

    for gain_p in y_pred:
        if gain_p>0.1:
            y_pos.append(1)
        elif gain_p<-0.1:
            y_pos.append(-1)
        else:
            y_pos.append(0)

    df_pred=pd.DataFrame()
    df_pred['symbol']=n50_symbols
    df_pred['price']=price_arr
    df_pred['gain%']=y_pred
    df_pred['abs gain%']=abs(y_pred)
    df_pred['pos']=y_pos
    df_pred= df_pred.sort_values(by='abs gain%',ascending=False).reset_index(drop=True)

    print(df_pred)
    x+=1

    #time.sleep(300)

        symbol     price     gain%  abs gain%  pos
0        NIFTY  21908.25  0.874721   0.874721    1
1   BAJAJ_AUTO   7325.00 -0.514945   0.514945   -1
2       MARUTI   9970.00 -0.478789   0.478789   -1
3   ULTRACEMCO   9822.00 -0.455097   0.455097   -1
4      DRREDDY   5692.00 -0.330653   0.330653   -1
5         NTPC    313.05  0.330290   0.330290    1
6    POWERGRID    239.65  0.322398   0.322398    1
7   BAJFINANCE   7639.00 -0.321174   0.321174   -1
8   APOLLOHOSP   5807.45 -0.309973   0.309973   -1
9    COALINDIA    381.85  0.273990   0.273990    1
10   BRITANNIA   5150.00 -0.271309   0.271309   -1
11  ADANIPORTS   1207.00  0.269880   0.269880    1
12        BPCL    458.10  0.245292   0.245292    1
13    AXISBANK   1119.05  0.242802   0.242802    1
14   TATASTEEL    135.45  0.242281   0.242281    1
15        SBIN    635.40  0.236503   0.236503    1
16    HINDALCO    582.00  0.226373   0.226373    1
17  TATAMOTORS    817.45  0.224547   0.224547    1
18  HEROMOTOCO   4369.00 -0.221

In [ ]:
tv = TvDatafeed()

# wanted_day_ind=12

df50=[]

for i in range(51):
    data_5_min = tv.get_hist(symbol=f'{n50_symbols[i]}',exchange='NSE',interval=Interval.in_5_minute,n_bars=2000)
    df=pd.DataFrame(data_5_min)
    df=df.reset_index()
    df['datetime']=pd.to_datetime(df['datetime']).dt.tz_localize("UCT")
    df['datetime']=df['datetime'].dt.tz_convert("Asia/Kolkata")
    df['year'] = df['datetime'].dt.year
    df['hour'] = df['datetime'].dt.hour
    df['minute'] = df['datetime'].dt.minute
    df50.append(df)


profit_m1=[]
profit_m2=[]

for wanted_day_ind in range(2,13):

    X_pred=[]
    y_pred=[]
    y_pos=[]
    price_arr=[]

    gain_test=[]
    change_arr=[]
    act_pos=[]

    for df in df50:
        #print(df)

        #####Indicators####
        df['ema200']=tb.EMA(df['close'],200)
        #aroon
        df["aroon"]=tb.AROONOSC(df['high'],df['low'],timeperiod=14)
        # #macd
        df["macd"]=tb.MACD(df['close'], fastperiod=12, slowperiod=26, signalperiod=9)[0]
        #bbands
        df['bbands_up']=tb.BBANDS(df['close'],5,2,2,0)[0]
        df['bbands_mid']=tb.BBANDS(df['close'],5,2,2,0)[1]
        df['bbands_down']=tb.BBANDS(df['close'],5,2,2,0)[2]
        #rsi
        df["rsi"]=tb.RSI(df['close'])


        df=df[201:].reset_index(drop=True)

        X_end_arr=[]
        for j in range(len(df.index)):
            if df['hour'][j]==15 and df['minute'][j]==15:
                X_end_arr.append(j)




        df_for_pred=df.iloc[X_end_arr[-wanted_day_ind]-70:X_end_arr[-wanted_day_ind]].drop(['symbol','datetime','year','hour','minute'],axis=1)
        #print(df_for_pred)

        df_for_pred = pd.DataFrame(imputer.fit_transform(df_for_pred), columns=df_for_pred.columns)

        X_pred.append(df_for_pred.values.flatten())

        # Price of stock at that instant
        price_arr.append(df['close'][X_end_arr[-wanted_day_ind]])

        # ind=end_arr[-1] ####################################################################
        g=(df['close'][X_end_arr[-wanted_day_ind]+9]-df['close'][X_end_arr[-wanted_day_ind]])*100/df['close'][X_end_arr[-wanted_day_ind]]
        gain_test.append(g)
        change=df['close'][X_end_arr[-wanted_day_ind]+9]-df['close'][X_end_arr[-wanted_day_ind]]
        change_arr.append(change)

    X_pred_norm=scaler.transform(X_pred)
    y_pred=svr_regressor.predict(X_pred_norm)
    #print(y_pred)

    for gain_p in y_pred:
        if gain_p>0.1:
            y_pos.append(1)
        elif gain_p<-0.1:
            y_pos.append(-1)
        else:
            y_pos.append(0)

    for gain_p in gain_test:
        if gain_p>0.1:
            act_pos.append(1)
        elif gain_p<-0.1:
            act_pos.append(-1)
        else:
            act_pos.append(0)

    df_test=pd.DataFrame()
    df_test['symbol']=n50_symbols
    df_test['price']=price_arr
    df_test['pred_gain%']=y_pred
    df_test['act_gain%']=gain_test
    df_test['abs pred gain%']=abs(y_pred)
    df_test['abs act gain%']=abs(df_test['act_gain%'])
    df_test['pred pos']=y_pos
    df_test['actual pos']=act_pos
    df_test['profit']=change_arr
    df_test= df_test.sort_values(by='abs pred gain%',ascending=False).reset_index(drop=True)

    # print(df_test)

    ############## Checking Profits##################

    cost=0
    profit1=0

    balance=10000

    df_top10=df_test.iloc[0:20]

    for i in range(10):
        symb=df_test['symbol'][i]

        if cost+df_test['price'][i]<=balance:
            quantity = int((balance/1.5) // df_test['price'][i])  # Allocate 2/3 of the balance for buying
            cost = quantity * df_test['price'][i]
            # print(df_test['symbol'][i])
            profit1+=quantity*df_test.loc[i,'pred pos']*df_test.loc[i,'profit']

    # print(f"balance1: {balance}")
    # print(f"Method1 profit: {profit1-0.0005*cost}")
    profit_m1.append(profit1-0.0005*cost)

    ######### Method2 ############

    profit=0
    cost=0
    balance=10000

    quantity=np.zeros(10)
    df_top10=df_test.iloc[0:10]

    for i in range(10):
        symb=df_test['symbol'][i]
        buy_percentage = 0.35  # Buy 35% of the available balance
        quantity[i] = int((balance * buy_percentage) // df_test['price'][i])
        cost = quantity[i] * df_test['price'][i]
        profit+=quantity[i]*df_test.loc[i,'pred pos']*df_test.loc[i,'profit']

    #Using the remaining balance

    for i in range(10):
        symb=df_test['symbol'][i]
        if cost+df_test['price'][i]<=balance:
            quantity[i]+=1
            cost+=df_test['price'][i]
            profit+=df_test.loc[i,'pred pos']*df_test.loc[i,'profit']

    balance-=cost

    df_top10['quantity']=quantity

    # print(f"balance2: {balance}")
    # print(f"Method2 profit: {profit-0.0005*cost}")
    profit_m2.append(profit-0.0005*cost)


print(profit_m1)
print(profit_m2)


<ipython-input-99-db2a0e3d1db1>:170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_top10['quantity']=quantity
<ipython-input-99-db2a0e3d1db1>:170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_top10['quantity']=quantity
<ipython-input-99-db2a0e3d1db1>:170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

[279.81402499999905, 287.81332499999957, -228.96362499999975, 95.49930000000032, 196.78004999999973, 194.4687499999998, 24.32727499999825, -250.65984999999895, 2.9592500000001873, 277.2964999999994, 24.284400000000705]
[162.40774999999942, 215.16792499999966, -134.41784999999985, 48.98160000000015, 132.607375, 106.34909999999988, 87.19992499999861, -137.62459999999908, 42.60072499999994, 122.16377499999949, 33.628300000000564]


<ipython-input-99-db2a0e3d1db1>:170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_top10['quantity']=quantity


In [ ]:
print(np.array(profit_m1).sum())
print(np.array(profit_m2).sum())

903.6193999999982
679.0640249999988
